In [1]:
# Azure Blob Storage access info
blob_account_name = "azureopendatastorage"
blob_container_name = "nyctlc"
blob_relative_path = "yellow"

# Construct connection path
wasbs_path = f'wasbs://{blob_container_name}@{blob_account_name}.blob.core.windows.net/{blob_relative_path}'
print(wasbs_path)

# Read parquet data from Azure Blob Storage path
blob_df = spark.read.parquet(wasbs_path)

StatementMeta(, ebdce48b-0899-459c-84a7-5fa363dc4c11, 3, Finished, Available)

wasbs://nyctlc@azureopendatastorage.blob.core.windows.net/yellow


In [2]:
# Declare file name    
file_name = "yellow_taxi"
# Construct destination path
output_parquet_path = f"abfss://ricaworkspace@onelake.dfs.fabric.microsoft.com/rica_lakehouse.Lakehouse/Files/RawData/{file_name}"
print(output_parquet_path)

# Load the first 1000 rows as a Parquet file
blob_df.limit(1000).write.mode("overwrite").parquet(output_parquet_path)

StatementMeta(, ebdce48b-0899-459c-84a7-5fa363dc4c11, 4, Finished, Available)

abfss://ricaworkspace@onelake.dfs.fabric.microsoft.com/rica_lakehouse.Lakehouse/Files/RawData/yellow_taxi


In [3]:
from pyspark.sql.functions import col, to_timestamp, current_timestamp, year, month

# Read the parquet data from the specified path
raw_df = spark.read.parquet(output_parquet_path)   

# Add dataload_datetime column with current timestamp
filtered_df = raw_df.withColumn("dataload_datetime", current_timestamp())

# Filter columns to exclude any NULL values in storeAndFwdFlag
filtered_df = filtered_df.filter(raw_df["storeAndFwdFlag"].isNotNull())

# Load the filtered data into a Delta table
table_name = "yellow_taxi"  # Replace with your desired table name
filtered_df.write.format("delta").mode("append").saveAsTable(table_name)

# Display results
display(filtered_df.limit(1))

StatementMeta(, ebdce48b-0899-459c-84a7-5fa363dc4c11, 5, Finished, Available)

SynapseWidget(Synapse.DataFrame, ab1f60f8-ebf5-4c11-bb12-1aec6fd23019)

In [4]:
from pyspark.sql.functions import col, to_timestamp, current_timestamp, year, month

# Read the parquet data from the specified path
raw_df = spark.read.parquet(output_parquet_path)    

# Add dataload_datetime column with current timestamp
opt_df = raw_df.withColumn("dataload_datetime", current_timestamp())

# Filter columns to exclude any NULL values in storeAndFwdFlag
opt_df = opt_df.filter(opt_df["storeAndFwdFlag"].isNotNull())

# Enable V-Order
spark.conf.set("spark.sql.parquet.vorder.enabled", "true")

# Enable automatic Delta optimized write
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "true")

# Load the filtered data into a Delta table
table_name = "yellow_taxi_opt"  # New table name
opt_df.write.format("delta").mode("append").saveAsTable(table_name)

# Display results
display(opt_df.limit(1))

StatementMeta(, ebdce48b-0899-459c-84a7-5fa363dc4c11, 6, Finished, Available)

SynapseWidget(Synapse.DataFrame, 6dfac04e-2ed3-4edf-88ea-273095ffeb7a)

In [5]:
# Load table into df
delta_table_name = "yellow_taxi"
table_df = spark.read.format("delta").table(delta_table_name)

# Create temp SQL table
table_df.createOrReplaceTempView("yellow_taxi_temp")

# SQL Query
table_df = spark.sql('SELECT * FROM yellow_taxi_temp')

# Display 10 results
display(table_df.limit(10))

StatementMeta(, ebdce48b-0899-459c-84a7-5fa363dc4c11, 7, Finished, Available)

SynapseWidget(Synapse.DataFrame, 9f7599df-3963-4b74-abff-4a7c496e1ad3)

In [6]:
# Load table into df
delta_table_name = "yellow_taxi_opt"
opttable_df = spark.read.format("delta").table(delta_table_name)

# Create temp SQL table
opttable_df.createOrReplaceTempView("yellow_taxi_opt")

# SQL Query to confirm
opttable_df = spark.sql('SELECT * FROM yellow_taxi_opt')

# Display results
display(opttable_df.limit(10))

StatementMeta(, ebdce48b-0899-459c-84a7-5fa363dc4c11, 8, Finished, Available)

SynapseWidget(Synapse.DataFrame, 9f9cca5f-0ccf-44b1-966f-d4deb25c9314)